In [8]:
import pandas as pd

In [9]:
# 读取数据并将standard_code列读取为字符串
df = pd.read_csv('data_with_embedding.xlsx', usecols=[1, 2, 3, 4], skiprows=1, nrows=276, header=None, names=['name', 'standard_code', 'description', 'embedding'], dtype={'standard_code': str})

# 对 standard_code 进行统一格式处理
def normalize_standard_code(code):
    parts = code.split('.')
    normalized_parts = [part.zfill(2) for part in parts]
    return '.'.join(normalized_parts)

In [10]:
df['standard_code'] = df['standard_code'].apply(normalize_standard_code)

print("读取的CSV数据:")
print(df)

df.to_csv('modified_data.csv', index=False)

读取的CSV数据:
       name standard_code                                        description  \
0     预防保健科            01  预防保健科主要对管辖区域的儿童保健、疾病预防、传染病管理、健康宣教，还有监督卫生监测等进行日...   
1     全科医疗科            02  全科医学科是一个面向社区与家庭，整合临床医学、预防医学、康复医学以及人文社会学科相关内容为一...   
2        内科            03         包括呼吸内科，消化内科，心血管内科，神经内科，肿瘤科，内分泌科，血液内科，传染病科等   
3    呼吸内科专业         03.01  呼吸内科专业是治疗呼吸系统疾病的一种科室，主要涉及肺结核、支气管哮喘、上呼吸道感染、肺气肿等...   
4    消化内科专业         03.02  消化内科科是一门专门研究人体消化系统及消化疾病的临床医学专业，涵盖了消化系统疾病的预防、诊断...   
..      ...           ...                                                ...   
271     彝医学         51.04  彝族人民长期同疾病作斗争的经验总结和智慧结晶。彝族有本民族文字，现在所能收集到的最早的彝族医...   
272     傣医学         51.05  傣医学主要研究傣药学、傣医诊疗法、傣医临床医疗技术等方面的基本知识和技能，运用傣医的理法方药...   
273      其他         51.06                                          民族医学科其他科室   
274  中西医结合科            52  西医结合科综合了现代医学与传统民族医学（包括中医药学、藏医、蒙医、维医等）的精髓，通过整合中...   
275  其他业务科室            69                                             其他业务科室   

                             

In [11]:
# 添加sub_description列
df.insert(loc=3,column='sub_description', value='')

df.to_csv('modified_data.csv', index=False)

In [12]:
# 初始化 sub_description 列
df['sub_description'] = ''

# 填充 sub_description 列
for i in range(len(df)):
    current_code = df.at[i, 'standard_code']
    sub_descriptions = []
    
    # 查找所有子级行
    for j in range(len(df)):
        child_code = df.at[j, 'standard_code']
        child_name = df.at[j, 'name']
        
        # 检查是否是当前行的子级，并且子级的标准编码应该以当前行的标准编码开头，并且层级比当前行的层级深1级
        if child_code.startswith(current_code + '.') and len(child_code.split('.')) == len(current_code.split('.')) + 1:
            if pd.notna(child_name):  # 只有 name 非空时才添加
                sub_descriptions.append(child_name)
    
    # 将子级行的 name 连接成字符串，并更新父级行的 sub_description 列
    if sub_descriptions:
        # 将当前行的 sub_description 列更新为子级 name 的逗号分隔字符串
        df.at[i, 'sub_description'] = ', '.join(sub_descriptions)


In [13]:
# 检查
# 打印部分 standard_code 的 sub_description
codes_to_print = ['03.04', '03.07', '03.08']
for code in codes_to_print:
    sub_description = df.loc[df['standard_code'] == code, 'sub_description'].values
    if len(sub_description) > 0:
        print(f"standard_code 为 {code} 的 sub_description 内容:")
        print(sub_description[0])
    else:
        print(f"未找到 standard_code 为 {code} 的数据。")

standard_code 为 03.04 的 sub_description 内容:
冠心病, 高血压病
standard_code 为 03.07 的 sub_description 内容:
糖尿病, 甲状腺疾病
standard_code 为 03.08 的 sub_description 内容:
风湿科, 痛风, 类风关, 系统性红斑狼疮


In [14]:
df.to_csv('modified_data.csv', index=False)
df[['standard_code', 'sub_description']].to_csv('sub_descriptions.csv', index=False)
print("修改后的CSV数据:")
print(df)

修改后的CSV数据:
       name standard_code                                        description  \
0     预防保健科            01  预防保健科主要对管辖区域的儿童保健、疾病预防、传染病管理、健康宣教，还有监督卫生监测等进行日...   
1     全科医疗科            02  全科医学科是一个面向社区与家庭，整合临床医学、预防医学、康复医学以及人文社会学科相关内容为一...   
2        内科            03         包括呼吸内科，消化内科，心血管内科，神经内科，肿瘤科，内分泌科，血液内科，传染病科等   
3    呼吸内科专业         03.01  呼吸内科专业是治疗呼吸系统疾病的一种科室，主要涉及肺结核、支气管哮喘、上呼吸道感染、肺气肿等...   
4    消化内科专业         03.02  消化内科科是一门专门研究人体消化系统及消化疾病的临床医学专业，涵盖了消化系统疾病的预防、诊断...   
..      ...           ...                                                ...   
271     彝医学         51.04  彝族人民长期同疾病作斗争的经验总结和智慧结晶。彝族有本民族文字，现在所能收集到的最早的彝族医...   
272     傣医学         51.05  傣医学主要研究傣药学、傣医诊疗法、傣医临床医疗技术等方面的基本知识和技能，运用傣医的理法方药...   
273      其他         51.06                                          民族医学科其他科室   
274  中西医结合科            52  西医结合科综合了现代医学与传统民族医学（包括中医药学、藏医、蒙医、维医等）的精髓，通过整合中...   
275  其他业务科室            69                                             其他业务科室   

                            